In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/NLP
!pip install import-ipynb
!pip install tensorboardX

# TensorBoard를 사용하면 손실 및 정확도와 같은 측정 항목을 추적 및 시각화하는 것, 모델 그래프를 시각화하는 것, 히스토그램을 보는 것, 이미지를 출력하는 것 등이 가능
import torch.utils.tensorboard as tensorboard 
import import_ipynb
import numpy as np

import os

from vision_model_final_sol import Mini_Xception
from vision_dataset_final_sol import create_train_dataloader, create_val_dataloader, create_test_dataloader

import torch
import torch.nn as nn
import torch.optim

from sklearn.metrics import accuracy_score
import random

/content/drive/MyDrive/NLP
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 33.5 MB/s 
importing Jupyter notebook from vision_model_final_sol.ipynb
importing Jupyter notebook from vision_dataset_final_sol.ipynb


In [3]:
seed_val = 2022
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# tensorboard 만들기
writer = tensorboard.SummaryWriter('checkpoint/tensorboard')
mini_xception = Mini_Xception()

def main(): # 코드가 실행될 main 함수를 만들어봅시다.
   
    # ========= dataloaders ===========
    train_dataloader = create_train_dataloader(root='./data', batch_size = 15)
    test_dataloader = create_val_dataloader(root='./data', batch_size=15)    
    # ========= load =========== 
    print("---Training---\n")

    for epoch in range(0, 5):
        # =========== train / validate ===========
        train_loss = train_one_epoch(mini_xception, train_dataloader, epoch)
        val_loss, accuracy= validate(mini_xception, test_dataloader, epoch)
        
        # tensorboard에 loss와 정확도 나타내기(매 epoch 마다)
        writer.add_scalar('train_loss',train_loss, epoch)
        writer.add_scalar('val_loss',val_loss, epoch)
        writer.add_scalar('accuracy',accuracy, epoch)
 
        # 매 epoch 마다 checkpoint로 model을 저장한다.
        if epoch % 1 == 0:
            checkpoint_state = {
                'mini_xception': mini_xception.state_dict(),
                "epoch": epoch
            }
            savepath = os.path.join('checkpoint/model_weights', f'weights_epoch_{epoch}.pth.tar')
            torch.save(checkpoint_state, savepath)
            print(f'\n\t*** Saved checkpoint in {savepath} ***\n')
    writer.close() # Tensorboard 닫기

def train_one_epoch(model, dataloader, epoch): # train 과정에서의 손실을 계산하는 함수 작성
    model.train()
    model.to(device)
    losses = [] # 손실 저장용 array
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(mini_xception.parameters(), lr=1e-3, weight_decay=1e-6)
    for images, labels in dataloader:
        images = images.to(device) # (batch, 1, 48, 48) -> images.shape[0]을 mini batch size로 나중에 설정
        labels = labels.to(device) # (batch,)
        emotions = model(images) # -> 예측값

        # (batch, 7, 1, 1) -> (batch, 7)
        emotions = torch.squeeze(emotions) # Returns a tensor with all the dimensions of input of size 1 removed.
        loss = criterion(emotions, labels) # 예측값과 label간 loss
        losses.append(loss.cpu().item())
        print(f'training @ epoch {epoch} .. loss = {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return np.mean(losses).item()

def validate(model, dataloader, epoch): 
  # 중간미션 1에서 with torch.no_grad(): 부분에 작성하였던 test accuracy를 구하는 방법을 떠올려보면 됩니다. 
  # train 과정에서의 손실도 중요하나 최종적으로 random data을 넣어서 test를 한 후의 정확도를 얻어야 합니다.
    model.eval() # dropout layer처럼 학습에는 사용하나 evaluation에서는 사용하지 않는 layer에 대해 .eval()을 먼저 설정해주면 그 layer은 동작하지 않습니다.
    model.to(device)
    losses = []
    total_pred = []
    total_labels = []

    criterion = nn.CrossEntropyLoss()
    with torch.no_grad(): # 학습이 x 모델 성능 평가 따라서 autograd하지 않음!!
        for images, labels in dataloader:
            mini_batch = images.shape[0]
            images = images.to(device)
            labels = labels.to(device)

            emotions = model(images)
            emotions = torch.squeeze(emotions)
            emotions = emotions.reshape(mini_batch, -1) # mini_batch로 shape 맞춤(계산위해서)

            loss = criterion(emotions, labels)            
            losses.append(loss.cpu().item())

            # # ============== Evaluation ===============
            # index of the max value of each sample (shape = (batch,))
            _, indexes = torch.max(emotions, axis=1)
            total_pred.extend(indexes.cpu().detach().numpy())
            total_labels.extend(labels.cpu().detach().numpy())
            
            print(f'validation loss = {loss.item()}')

        val_loss = np.mean(losses).item()
        accuracy = accuracy_score(total_labels, total_pred)
        
        
        print(f'Val loss = {val_loss} .. Accuracy = {accuracy:.4f} ')

        return val_loss, accuracy

if __name__ == "__main__": # 앞서 작성한 main 함수 실행
    main()


training @ epoch 0 .. loss = 1.9318301677703857
training @ epoch 0 .. loss = 1.5467197895050049
training @ epoch 0 .. loss = 1.3518636226654053
training @ epoch 0 .. loss = 1.6391366720199585
training @ epoch 0 .. loss = 1.7041922807693481
training @ epoch 0 .. loss = 1.642380952835083
training @ epoch 0 .. loss = 2.0257010459899902
training @ epoch 0 .. loss = 1.5268796682357788
training @ epoch 0 .. loss = 1.7521225214004517
training @ epoch 0 .. loss = 1.4879364967346191
training @ epoch 0 .. loss = 1.9769564867019653
training @ epoch 0 .. loss = 1.823913335800171
training @ epoch 0 .. loss = 1.6838853359222412
training @ epoch 0 .. loss = 1.9644886255264282
training @ epoch 0 .. loss = 1.8103220462799072
training @ epoch 0 .. loss = 1.6184837818145752
training @ epoch 0 .. loss = 1.7821850776672363
training @ epoch 0 .. loss = 1.798424243927002
training @ epoch 0 .. loss = 1.8689876794815063
training @ epoch 0 .. loss = 1.8685249090194702
training @ epoch 0 .. loss = 1.643312573432

KeyboardInterrupt: ignored

In [5]:
# 저장해놓은 텐서보드 로드하기(in colab) - loss와 accuracy의 경향 확인
%load_ext tensorboard
%tensorboard --logdir {'checkpoint/tensorboard'}

<IPython.core.display.Javascript object>